In [ ]:
from pathlib import Path

import matplotlib.dates as mdates
import pandas as pd
from matplotlib.ticker import FixedFormatter, FixedLocator

from analytics.app.data.load import list_pipelines
from analytics.app.data.transform import dfs_models_and_evals, logs_dataframe
from analytics.plotting.common.metric_over_time import plot_metric_over_time
from analytics.plotting.common.save import save_plot

%load_ext autoreload
%autoreload 2

In [ ]:
pipelines_dir = Path(
    "/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/arxiv/10_baselines_time"
)
assert pipelines_dir.exists()

In [ ]:
pipelines = list_pipelines(pipelines_dir)
max_pipeline_id = max(pipelines.keys())
pipelines

In [ ]:
from analytics.app.data.load import load_pipeline_logs

pipeline_logs = {p_id: load_pipeline_logs(p_id, pipelines_dir) for (p_id, (_, p_path)) in pipelines.items()}

In [ ]:
# mode:
pipeline_id = 267

# doesn't do anything unless include_composite_model = True
composite_model_variant = "currently_active_model"

patch_yearbook = True
dataset_id = "arxiv_kaggle_test"
eval_handler = "periodic-current"
metric = "Accuracy"
include_composite_model = True

# Wrangle data

In [ ]:
pipeline_log = pipeline_logs[pipeline_id]
pipeline_ref = f"{pipeline_id}".zfill(len(str(max_pipeline_id))) + f" - {pipelines[pipeline_id][0]}"

df_all = logs_dataframe(pipeline_log, pipeline_ref)

df_logs_models, _, df_eval_single = dfs_models_and_evals(
    # subtracting would interfere with yearbook patching
    pipeline_log,
    df_all["sample_time"].max(),
    pipeline_ref,
)

df_adjusted = df_eval_single


df_adjusted = df_adjusted[
    (df_adjusted["dataset_id"] == dataset_id)
    & (df_adjusted["eval_handler"] == eval_handler)
    & (df_adjusted["metric"] == metric)
]

# in percent (0-100)
df_adjusted["value"] = df_adjusted["value"] * 100
df_adjusted

In [ ]:
df_adjusted["pipeline_ref"].unique()

In [ ]:
# df_adjusted["dataset_id"].unique()
df_adjusted[df_adjusted["dataset_id"] == "yearbook-test"]["pipeline_ref"].unique()

In [ ]:
df_adjusted = df_adjusted.sort_values(by=["interval_center"])

In [ ]:
# Add composite model


pipeline_composite_model = df_adjusted[df_adjusted[composite_model_variant]]
pipeline_composite_model["model_idx"] = -1  # "00-pipeline-composite-model"
# number_digits = len(str(df_adjusted["model_idx"].max()))
# df_adjusted["model_idx"] = df_adjusted["model_idx"].astype(str).str.zfill(number_digits)
df_adjusted = pd.concat([df_adjusted, pipeline_composite_model])

# df_composite = df_adjusted[df_adjusted[composite_model_variant]]
# df_composite

In [ ]:
min_model = 1
max_model = df_adjusted["model_idx"].max()
thresholds_early_models = (6, 11)
threshold_late_models = 49

reduced = df_adjusted.copy()
reduced["early_model"] = (reduced["model_idx"] <= thresholds_early_models[1]) & (
    reduced["model_idx"] >= thresholds_early_models[0]
)
reduced["late_model"] = reduced["model_idx"] >= threshold_late_models
reduced["composite_model"] = reduced["model_idx"] == -1
reduced = reduced[reduced["early_model"] | reduced["late_model"] | reduced["composite_model"]]
# assert len(reduced[reduced["early_model"]]["model_idx"].unique()) == 10
# assert len(reduced[reduced["late_model"]]["model_idx"].unique()) == 10

In [ ]:
# if early_model --> "early model", if late_model --> "late model", else "composite model"
def mapper(x):
    if x < 0:
        return "Cur. Active\nComposite Model"
    if (x >= thresholds_early_models[0]) & (x <= thresholds_early_models[1]):
        return f"Early Models ({thresholds_early_models[0]} - {thresholds_early_models[1]})"
    if x >= threshold_late_models:
        return f"Late Models ({threshold_late_models} - {max_model})"
    return "hide"


reduced["model_type"] = reduced["model_idx"].apply(mapper)
reduced["model_type"].unique()

In [ ]:
reduced

# Create Plot

In [ ]:
fig = plot_metric_over_time(
    reduced,
    x="interval_center",
    y="value",
    hue="model_type",
    style="model_type",
    width_factor=0.85,
    height_factor=0.65,
    legend_label="Model Type",
    small_legend_fonts=True,
    # x_date_locator=mdates.YearLocator(20),
    # x_date_formatter=mdates.DateFormatter("%Y"),  # %b\n
    x_date_locator=FixedLocator([mdates.date2num(pd.Timestamp(d)) for d in ["2000-01-01", "2009-01-01", "2018-01-01"]]),
    x_date_formatter=FixedFormatter([str(year) for year in ["Jan 2000", "Jan 2009", "Jan 2018"]]),
    xlim=(pd.Timestamp("1995-01-01"), pd.Timestamp("2024-09-01")),
    ylim=(-20, 75),
    y_ticks=[0, 20, 40, 60],
    x_label="Evaluation Year",
    y_label="Accuracy (%)",
    markers=False,
)

save_plot(fig, "simple_arxiv_26w_begin_vs_start_vs_composite")